In [ ]:
import hvplot.xarray  # noqa: F401
import numpy as np
from fsspec.implementations.http import HTTPFileSystem
from dea_tools.spatial import xr_vectorize
import geohash

from emit_tools import emit_xarray
from utils import get_rgb_dataset, get_earthdata_token

In [ ]:
# See README.md for instructions on how to get an Earthdata token
token = get_earthdata_token()

In [ ]:
%%time
# Loading data can take around 3-4 minutes on a 100 Mbps connection

# Refer to the README.md for instructions on how to find granule IDs
granule = "EMIT_L2A_RFL_001_20230316T045211_2307503_006" # Canberra

s3_url = "s3://lp-prod-protected/EMITL2ARFL.001/" + granule + "/" + granule + ".nc"
http_url = s3_url.replace("s3://", "https://data.lpdaac.earthdatacloud.nasa.gov/")

fs = HTTPFileSystem(headers={
    "Authorization": f"bearer {token}"
})
ds = emit_xarray(fs.open(http_url))
ds

In [ ]:
# Clean up empty bands.
ds = ds.fillna(np.nan).where(ds.reflectance!=-0.01)

In [ ]:
# Create a water layer
high = ds.reflectance.sel(bands=450, method="nearest")
low = ds.reflectance.sel(bands=1275, method="nearest")

water = ((high - low) / (high + low)) > 0.0
ds["water"] = water.fillna(float("nan")).where(water)

In [ ]:
ds.water.hvplot(aspect="equal")

In [ ]:
def add_geohash(row):
    return geohash.encode(row.geometry.centroid.y, row.geometry.centroid.x, precision=9)

# Create polygons from the water layer
water_polygons = xr_vectorize(ds.water, crs="epsg:4326", mask=ds.water.values==1)
water_polygons["area"] = water_polygons.to_crs("epsg:3577").area / 10000

# Drop geopandas rows where the area is less than 1 hectare
water_polygons = water_polygons.drop(water_polygons[water_polygons['area'] < 1].index)

# Compute a geohash for each polygon at level 9
water_polygons["geohash"] = water_polygons.apply(add_geohash, axis=1)

# Show us what we've got
water_polygons.plot()

In [ ]:
# View the water layer on an interactive map
import folium
import odc.geo.xr
import geopandas as gpd

m = folium.Map(control_scale=True, tiles=None)

# ds.water.odc.add_to(m, name="Water", cmap="Blues")

for _, r in water_polygons.iterrows():
    # Without simplifying the representation of each borough,
    # the map might not be displayed
    sim_geo = gpd.GeoSeries(r["geometry"]).simplify(tolerance=0.0001)
    geo_j = sim_geo.to_json()
    geo_j = folium.GeoJson(data=geo_j, style_function=lambda x: {"fillColor": "blue", "Color": "blue"})
    folium.Popup(f"geohash: {r['geohash']}\narea: {r['area'] / 10000:.3f} Ha").add_to(geo_j)
    geo_j.add_to(m)
m

# Zoom map
m.fit_bounds(ds.odc.map_bounds())

tile = folium.TileLayer(
    tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
    attr = 'Esri',
    name = 'Esri Satellite',
    control = True
).add_to(m)

folium.LayerControl().add_to(m)
display(m)

In [ ]:
# Export water polygons as geojson
water_polygons.to_file("water_polygons.geojson", driver="GeoJSON")